In [32]:
import math
import json
import numpy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from causality.inference.search import IC
from causality.inference.independence_tests import *

from matplotlib.cm import get_cmap
from matplotlib.colors import rgb2hex

from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider
from bokeh.models.glyphs import *
from bokeh.models.arrow_heads import *
from bokeh.models.annotations import *
from bokeh.models.graphs import from_networkx

In [2]:
df = pd.read_csv("data/wdvp_stats.tsv", 
                 sep="\t", 
                 header=0, 
                 skiprows=range(1, 5), 
                 index_col=0, 
                 thousands=',',
                 na_values=["-"])
df.drop("ISO Country code", axis=1, inplace=True)
df.dropna(axis=1, how="all", inplace=True)

In [3]:
df.head()

,population,surface area (Km2),GINI index,happy planet index,human development index,world happiness report score,sustainable economic development assessment (SEDA),GDP (billions PPP),GDP per capita (PPP),GDP growth (annual %),...,regulatory quality,rule of law,control of corruption,judicial effectiveness score,government integrity score,property rights score,tax burden score,overall economic freedom score,financial freedom score,women MPs (% of all MPs)
indicator,,,,,,,,,,,,,,,,,,,,,
Afghanistan,36000000,652230,NaN,20.2,0.498,2.66,NaN,64.1,1919.0,1.5,...,-1.3,-1.57,-1.52,28.2,26.2,17.9,91.8,51.3,10.0,27.7
Albania,2900000,27398,29.0,36.8,0.785,4.64,53.1,34.2,11840.0,2.6,...,0.2,-0.40,-0.42,25.4,39.9,54.1,85.1,64.5,70.0,27.9
Algeria,41000000,2381740,35.3,33.3,0.754,5.25,45.8,612.5,15027.0,3.7,...,-1.2,-0.86,-0.61,35.2,29.0,27.8,74.0,44.7,30.0,25.8
Andorra,77000,468,NaN,NaN,0.858,NaN,NaN,NaN,NaN,NaN,...,1.2,1.60,1.24,NaN,NaN,NaN,NaN,NaN,NaN,32.1
Angola,30000000,1246700,42.7,NaN,0.581,NaN,28.4,187.3,6844.0,3.0,...,-1.0,-1.10,-1.41,25.4,18.9,36.0,82.4,48.6,40.0,38.2


In [4]:
# define the variable types: 'c' is 'continuous'
variables = [
    "GINI index",
    "happy planet index",
    "human development index",
    "world happiness report score",
    "sustainable economic development assessment (SEDA)",
    "GDP per capita (PPP)",
    "GDP growth (annual %)",
    "health expenditure  % of GDP",
    "health expenditure  per person",
    "education expenditure % of GDP",
    "education expenditure  per person ",
    "school life expectancy (YEARS)",
    "unemployment (%)",
    "government spending score",
    "government expenditure (% of GDP)",
    "political rights score ",
    "civil liberties score ",
    "political stability & absence of violence",
    "government effectiveness",
    "regulatory quality",
    "rule of law",
    "control of corruption",
    "judicial effectiveness score",
    "government integrity score",
    "property rights score",
    "tax burden score",
    "overall economic freedom score",
    "financial freedom score",
    "women MPs (% of all MPs)"
]

variable_types = {v: "c" for v in variables}

# run the search
ic_algorithm = IC(RobustRegressionTest)
causal_graph = ic_algorithm.search(df.fillna(df.mean()), variable_types)

In [5]:
D = nx.DiGraph()
D.add_nodes_from(causal_graph.nodes())

for e in causal_graph.edges(data=True):
    n1, n2 = e[:2]
    data = e[2]
    arrows = data["arrows"]
    if not arrows:
        D.add_edge(n2, n1, marked=data["marked"], directed=False, both=None)
    elif (n1 in arrows and n2 in arrows):
        D.add_edge(n2, n1, marked=data["marked"], directed=True, both=True)
    elif n1 in arrows:
        D.add_edge(n2, n1, marked=data["marked"], directed=True, both=False)
    elif n2 in arrows:
        D.add_edge(n1, n2, marked=data["marked"], directed=True, both=False)

pos = nx.circular_layout(D)

In [176]:
from networkx.readwrite.json_graph import node_link_data
import pickle

with open("data/graph", "wb") as f:
    f.write(pickle.dumps(node_link_data(D)))

with open("data/pos", "wb") as f:
    f.write(pickle.dumps(pos))

In [168]:
RADIUS = 0.15
STEPS = 1000
OUTLINE = "#383838"

def dist(l1, l2):
    x1, y1 = l1
    x2, y2 = l2
    return ((y2 - y1)**2 + (x2 - x1)**2)**0.5

def bezier(l1, l2, b):
    x1, y1 = l1
    x2, y2 = l2
    d = dist(l1, l2)
    t = b * (1 + d)
    steps = [i/STEPS for i in range(STEPS)]
    xs = [(1-s)**t*x1 + s**t*x2 for s in steps]
    ys = [(1-s)**t*y1 + s**t*y2 for s in steps]
    return xs, ys

def colormap(num):
    colors = ["#f7c031", "#ef4837", "#91b5bb", "#526354", "#fecacb"]
    return list(colors * 100)[:num]

def nearest_offset(xs, ys, centroid):
    for i, (x, y) in enumerate(zip(xs[::-1], ys[::-1])):
        if dist(centroid, (x, y)) > RADIUS-0.075:
            break
    return i

# define plot
plot = figure(title="Causal Discovery on World Development Indices", x_range=(-1.7, 1.7), y_range=(-1.7, 1.7),
              tools="pan,wheel_zoom,box_zoom,reset,save", plot_width=975, plot_height=975, match_aspect=True)

plot.background_fill_color = "#e0e0e0"
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None
plot.axis.visible = False

graph = GraphRenderer()

# add nodes
graph.node_renderer.data_source.add(list(pos.keys()), "index")
graph.node_renderer.data_source.add(colormap(len(pos)), "color")
graph.node_renderer.glyph = Ellipse(height=RADIUS, width=RADIUS, fill_color="color", line_color=OUTLINE, line_width=4)

# add directed edges
graph.edge_renderer.data_source.data = dict(start=[], end=[], xs=[], ys=[], edge_color=[])

for e in D.edges():
    n1, n2 = e
    l1, l2 = pos[n1], pos[n2]
    xs, ys = bezier(l1, l2, 1)
    graph.edge_renderer.data_source.data["start"].append(n1)
    graph.edge_renderer.data_source.data["end"].append(n2)
    graph.edge_renderer.data_source.data["xs"].append(xs)
    graph.edge_renderer.data_source.data["ys"].append(ys)
    graph.edge_renderer.data_source.data["edge_color"].append(OUTLINE)

graph.edge_renderer.glyph = MultiLine(line_width=3, line_color="edge_color")
    
for e, xs, ys in zip(D.edges(data=True),
                     graph.edge_renderer.data_source.data["xs"],
                     graph.edge_renderer.data_source.data["ys"]):
    n1, n2, data = e
    l1, l2 = pos[n1], pos[n2]
    x1, y1 = l1
    x2, y2 = l2
    if data["directed"]:
        os = nearest_offset(xs, ys, l2)
        arrow = VeeHead(fill_color=OUTLINE, line_color=None, size=15)
        plot.add_layout(
            Arrow(end=arrow, x_start=xs[-os-1], y_start=ys[-os-1], x_end=xs[-os], y_end=ys[-os], line_color=None)
        )
        if data["both"]:
            plot.add_layout(
                Arrow(end=arrow, x_start=xs[os+1], y_start=ys[os+1], x_end=xs[os], y_end=ys[os], line_color=None)
            )

# add labels
p_ind = np.linspace(0, 1-1/len(pos), len(pos)) * np.pi * 2
xr = 1.1 * np.cos(p_ind)
yr = 1.1 * np.sin(p_ind)
rad = np.arctan2(yr, xr)
plot.text(xr, yr, list(D.nodes()), angle=rad,
    text_font_size="9pt", text_align="left", text_baseline="middle")

# render
graph.layout_provider = StaticLayoutProvider(graph_layout=pos)
plot.renderers.append(graph)
output_notebook()
show(plot)

Loading BokehJS ...